In [2]:
# 使用 python-dotenv 加载 .env 文件
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.messages import HumanMessage, SystemMessage

import os

# 加载当前目录下的 .env 文件
load_dotenv(override=True)

# 获取环境变量
api_key = os.getenv("API_KEY")
base_url = os.getenv("BASE_URL")
model_name = os.getenv("MODEL_NAME")


# 初始化模型
model = ChatOpenAI(
    base_url=base_url,
    model=model_name,
    api_key=api_key,
    temperature=1,
)

In [ ]:
from typing import List, TypedDict, Annotated, Tuple, Literal, Optional, Any, Dict
from langchain.tools import tool, ToolRuntime
from langchain.messages import HumanMessage, SystemMessage, ToolMessage
from prompt import (
    CODE_GENERATOR_SYSTEM_PROMPT,
    CODE_INTERPRETER_AGENT_PROMPT,
    RESULT_ANALYZER_SYSTEM_PROMPT,
)
from utils import extract_python_code
from langchain.agents import create_agent, AgentState
import json
from langgraph.types import Command

# TODO 还可以继续优化，需要在execute_code那把添加一个llm，来提取每次都data_context，里面的列schema等等，做总结


# 构建sub agent 作为bi数据分析 然后会集成到主agent的tool中
class BIAgentState(AgentState):
    current_task: str  # 目标
    data_context: Dict[str, Any]  # 数据上下文（文件路径、schema等）
    generated_code: str  # 当前生成的代码
    execution_result: str  # 执行结果
    error_message: str  # 错误信息
    analysis: str  # 分析结论
    iteration_count: int  # 重试次数
    max_iterations: int  # 最大重试次数


@tool
def generate_code(task_description: str, runtime: ToolRuntime) -> Command:
    """
    生成Python数据分析代码。

    这个工具会调用LLM生成高质量的、可执行的Python代码。

    Args:
        task_description: 要完成的任务描述，例如："分析data.csv的销售趋势"

    Returns:
        生成的Python代码字符串（不含markdown代码块标记）

    Examples:
        >>> # 生成代码
        >>> code = generate_python_code(
        ...     task_description="统计data.csv的基本信息",
        ... )
    """

    data_context = runtime.state.get("data_context")
    previous_error = runtime.state.get("error_message")

    err_prompt = ""
    data_context_prompt = ""

    if data_context:
        try:
            context_dict = (
                json.loads(data_context)
                if isinstance(data_context, str)
                else data_context
            )
            context_str = json.dumps(context_dict, ensure_ascii=False, indent=2)
            data_context_prompt = f"**数据上下文:**\n```json\n{context_str}\n```\n"
        except:
            data_context_prompt = f"**数据上下文:**\n{data_context}\n"

    if previous_error:
        err_prompt = f"""
**⚠️ 之前的代码执行失败了！错误信息如下:**
```
{previous_error}
```

**请仔细分析错误原因，生成修复后的代码。**
常见修复方法:
- FileNotFoundError: 检查文件路径，使用绝对路径或确认相对路径正确
- KeyError: 先打印df.columns查看实际列名，不要假设列名
- ValueError: 检查数据类型，必要时进行类型转换
- ImportError: 确保导入了所有必要的库
"""

    user_message = f"""
        **任务描述:**
        {task_description}
        {data_context_prompt}
        {err_prompt}
        **输出要求:**
        1. 只输出Python代码，不要有任何解释文字
        2. 代码必须放在 ```python 代码块内
        3. 代码要完整、可直接执行
        4. 使用print()输出关键信息
    """

    print("****generate_code 用户输入：***** \n", user_message)
    print("\n")

    # 构造messages
    messages = [
        SystemMessage(content=CODE_GENERATOR_SYSTEM_PROMPT),
        HumanMessage(content=user_message),
    ]

    # 调用llm
    model = ChatOpenAI(
        base_url=base_url,
        model=model_name,
        api_key=api_key,
        temperature=0.2,  # 代码生成使用较低温度，保证稳定性
    )

    result = model.invoke(messages)

    raw_content = result.content
    # 提取python代码（去除markdown标志）
    code = extract_python_code(raw_content)
    if not code:
        # 如果没有提取到代码块，可能LLM直接返回了代码
        code = raw_content.strip()

    print("生成的代码:\n", code)
    print("\n")

    return Command(
        update={
            "generated_code": code,
            "messages": [
                ToolMessage(
                    content=code,
                    tool_call_id=runtime.tool_call_id,
                )
            ],
        }
    )


@tool
def execute_code(runtime: ToolRuntime):
    """运行python脚本

    不需要任何参数！直接从 State 读取生成的代码并执行。

    Returns:
        执行结果描述
    """
    import subprocess

    code = runtime.state.get("generated_code")

    print("获得的代码:\n", code)
    print("\n")

    if not code:
        return Command(
            update={
                "error_message": "没有可执行的代码",
                "messages": [
                    ToolMessage(
                        content="❌ 错误：没有可执行的代码。请先调用 generate_code。",
                        tool_call_id=runtime.tool_call_id,
                    )
                ],
            }
        )

    result = subprocess.run(
        ["python", "-c", code], capture_output=True, text=True, timeout=10
    )

    if result.returncode == 0:
        return Command(
            update={
                "execution_result": result.stdout,
                "error_message": "",  # 清空错误信息
                "messages": [
                    ToolMessage(
                        content=f"✅ 代码执行成功\n\n{result.stdout}",
                        tool_call_id=runtime.tool_call_id,
                    )
                ],
            }
        )
    else:
        return Command(
            update={
                "error_message": result.stderr,
                "messages": [
                    ToolMessage(
                        content=f"❌ 代码执行失败\n\n错误信息：\n{result.stderr}",
                        tool_call_id=runtime.tool_call_id,
                    )
                ],
            }
        )


@tool
def analyze_results(runtime: ToolRuntime):
    """
    分析代码执行结果，生成用户友好的数据分析报告。

    这个工具专注于分析代码的执行输出，将技术性结果转化为业务洞察。

    不需要任何参数，从 State 中读取所有需要的信息（task、output、data_context）。

    Returns:
        Markdown格式的数据分析报告

    Examples:
        >>> report = analyze_results()
        >>> print(report)
        # 销售数据分析报告
        ...

    Note:
        为了让分析更准确，生成的代码应该在输出中包含：
        - 清晰的任务描述（"正在分析销售趋势..."）
        - 数据处理步骤说明
        - 图表类型和保存路径（如果生成了图表）
        这样分析工具无需查看代码就能理解做了什么。
    """
    # 初始化LLM
    from dotenv import load_dotenv
    import os

    load_dotenv(override=True)

    llm = ChatOpenAI(
        base_url=os.getenv("BASE_URL"),
        model=os.getenv("MODEL_NAME"),
        api_key=os.getenv("API_KEY"),
        temperature=0.7,  # 分析报告可以稍微有创造性
    )

    original_task = runtime.state.get("current_task", "数据分析任务")
    data_context = runtime.state.get("data_context", {})
    execution_output = runtime.state.get("execution_result", "")

    # 构建用户消息（不再包含代码）
    user_message = f"""
请基于以下信息，撰写一份专业的数据分析报告。

## 上下文信息

### 1. 用户原始任务
{original_task}

### 2. 数据上下文
{data_context or "无额外上下文"}

### 3. 代码执行输出
```
{execution_output}
```

## 分析要求

请按照以下结构撰写报告（使用Markdown格式）：

1. **数据概览**: 解读统计数据，说明数据规模、质量、基本特征
2. **核心发现**: 3-5个最重要的洞察，每个洞察要有数据支撑和业务解读
3. **图表解读**: 如果输出中提到生成了图表，解释图表的作用和意义
4. **结论**: 直接回答用户的原始问题
5. **建议**: 基于数据提供可行的业务建议

## 写作原则

- 用业务语言，避免技术术语
- 每个结论都要有数据支撑
- 关注"So What"（数据的业务含义）
- 简洁有力，一段话表达一个核心观点
- 提供具体、可操作的建议

请开始撰写分析报告：˝
"""

    print("报告用户输入:\n", user_message)
    print("\n")

    # 调用LLM
    messages = [
        SystemMessage(content=RESULT_ANALYZER_SYSTEM_PROMPT),
        HumanMessage(content=user_message),
    ]

    response = llm.invoke(messages)
    analysis_report = response.content

    print("报告结果：\n", analysis_report)
    print("\n")

    return Command(
        update={
            "analysis": analysis_report,
            "messages": [
                ToolMessage(content=analysis_report, tool_call_id=runtime.tool_call_id)
            ],
        }
    )


bi_agent = create_agent(
    model=model,
    tools=[generate_code, execute_code, analyze_results],
    system_prompt=CODE_INTERPRETER_AGENT_PROMPT,
    state_schema=BIAgentState,
)


# response = bi_agent.invoke(
#     {
#         "messages": [
#             {
#                 "role": "user",
#                 "content": f"""请完成以下数据分析任务：

# ## 用户查询
# 深度分析一下 ./data.csv中的数据,结合可视化

# ## 数据上下文
# ```json
# 无
# ```

# 请按照标准流程开始执行任务。
# """,
#             }
#         ],
#         "data_context": {"file_path": "./data.csv"},
#     },
# )

# print("bi agent最终结果:", response)

# response = bi_agent.stream(
#     {
#         "messages": [
#             {
#                 "role": "user",
#                 "content": f"""请完成以下数据分析任务：

# ## 用户查询
# 分析一下 ./data.csv中的数据

# ## 数据上下文
# ```json
# 无
# ```

# 请按照标准流程开始执行任务。
# """,
#             }
#         ]
#     },
#     stream_mode="messages",
# )

# for token, metadata in response:
#     print(f"node: {metadata['langgraph_node']}")
#     print(f"content: {token.content_blocks}")
#     print("\n")

****generate_code 用户输入：***** 
 
        **任务描述:**
        深度分析./data.csv中的数据，包括数据基本信息、统计描述、缺失值检查，并结合可视化图表展示数据分布和关系
        **数据上下文:**
```json
{
  "file_path": "./data.csv"
}
```

        
        **输出要求:**
        1. 只输出Python代码，不要有任何解释文字
        2. 代码必须放在 ```python 代码块内
        3. 代码要完整、可直接执行
        4. 使用print()输出关键信息
    


生成的代码:
 import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

def analyze_data_comprehensive():
    file_path = './data.csv'
    
    # 步骤1: 检查文件是否存在
    print("=" * 70)
    print("步骤1: 检查数据文件")
    print("=" * 70)
    
    if not os.path.exists(file_path):
        print(f"❌ 错误: 文件 {file_path} 不存在")
        return None
    
    print(f"✓ 文件存在: {file_path}")
    
    try:
        # 步骤2: 加载数据
        print("\n" + "=" * 70)
        print("步骤2: 加载数据")
        print("=" * 70)
        
        df = pd.read_csv(file_path)
        print(f"✓ 数据加载成功")
        print(f"  - 数据形状: {df.shape[0]} 行, {df.shape[1]} 列")
        
   

In [ ]:
# 基于plan and execute 作为主agent
from pydantic import BaseModel, Field
from typing import List, TypedDict, Annotated, Tuple, Literal, Optional
import operator
from prompt import PLANNER_SYSTEM_PROMPT, EXECUTOR_SYSTEM_PROMPT, REPLAN_SYSTEM_PROMPT
from langchain.agents import create_agent


@tool
def data_analysis(task_description: str, file_path: str) -> str:
    """
    数据分析

    自动生成并执行 Python 代码进行数据分析，具备错误自动修复能力。
    从 State 中读取数据上下文，不需要 LLM 重复传参。

    Args:
        task_description: 分析任务描述（LLM 决策）
        file_path: 要分析的数据所在路径

    Returns:
        分析报告

    Examples:
        >>> # 在 Plan-and-Execute 中使用
        >>> result = data_analysis(
        ...     {"task_description": "分析销售数据",file_path:"./data.csv"},
        ... )
    """

    response = bi_agent.invoke(
        {
            "messages": [
                {
                    "role": "user",
                    "content": f"""请完成以下数据分析任务：

        ## 用户查询
        {task_description}

        ## 数据上下文
        ```json
        无
        ```

        请按照标准流程开始执行任务。
        """,
                }
            ],
            "data_context": {"file_path": file_path},
        },
    )

    return response.get("analysis")


class AgentState(TypedDict):
    user_query: str  # 用户问题
    steps: List[str]  # 将要执行的计划
    past_steps: Annotated[
        List[Tuple], operator.add
    ]  # 已经执行了的步骤 [step,result]元组
    response: str


class Plan(BaseModel):
    """执行计划"""

    steps: List[str] = Field(description="要遵循的不同步骤，应按顺序排列")


def plan_node(state: AgentState):

    # 构造messages
    messages = [
        SystemMessage(content=PLANNER_SYSTEM_PROMPT),
        HumanMessage(
            f"""
**用户问题：** "{state['steps']}"

**文件信息：**
- **File Path**: `./data.csv`
- **Known Schema**: 无 
(如果 Known Schema 为空，请先制定计划去获取它。)
"""
        ),
    ]

    planer = model.with_structured_output(Plan)

    result = planer.invoke(messages)

    return {steps: result.steps}


def execute_node(state: AgentState):
    # 拿到当前要执行的步骤
    current_plan = state["steps"][0]
    # 拿到之前完成了的步骤和结果
    history_steps = "暂无历史记录(这是第一步)"

    if state["past_steps"]:
        history_steps = "\n".join(
            f"步骤:{step}\n结果:{result}\n" for step, result in state["past_steps"]
        )
    # 构造messages
    messages = [
        SystemMessage(content=EXECUTOR_SYSTEM_PROMPT),
        HumanMessage(
            content=f"""请利用工具执行以下具体任务：

### 1. 项目背景 (Context)
**总体目标**: {state['steps']}

### 2. 参考资料 (Execution History)
*以下是前序步骤的执行结果，请从中提取你需要的参数（如ID、链接、关键数据），不要重复做这些工作：*
----------------------------------------
{history_steps}
----------------------------------------

### 3. 当前任务 (Current Mission)
**请立即执行此步骤**: {current_plan}
"""
        ),
    ]

    executor = create_agent(
        tools=[],
        model=model,
    )

    # 调用执行当前节点
    response = executor.invoke(input={"messages": messages})

    return {"past_steps": [(current_plan, response["messages"][-1].content)]}


class Replan(BaseModel):
    """重规划者的决策结果"""

    status: Literal["done", "continue"] = Field(
        description="如果是 'done'，表示任务已完成。如果是 'continue'，表示需要执行新计划。"
    )
    new_plan: Optional[List[str]] = Field(
        default=None,
        description="如果 status 是 'continue'，这里必须包含剩余的、更新后的步骤列表。",
    )
    final_response: Optional[str] = Field(
        default=None,
        description="如果 status 是 'done'，这里必须包含回答用户问题的最终完整回复。",
    )


def replan_node(state: AgentState):

    # 拿到所有还没执行的步骤
    current_plan_list = "无"

    if state["steps"]:
        current_plan_list = "".join(f"{step}," for step in state["steps"])

    # 拿到之前执行完了的步骤和结果
    history_steps = "暂无历史记录"

    if state["past_steps"]:
        history_steps = "\n".join(
            f"步骤:{step}\n结果:{result}\n" for step, result in state["past_steps"]
        )

    # 构造messages
    messages = [
        SystemMessage(content=REPLAN_SYSTEM_PROMPT),
        HumanMessage(
            content=f"""
        请基于以下最新的项目状态进行决策：

# 1. 原始目标 (Goal)
{state['user_query']}

# 2. 当前剩余计划 (Current Plan)
{current_plan_list}

# 3. 执行历史档案 (Execution History)
----------------------------------------
{history_steps}
----------------------------------------
"""
        ),
    ]

    replaner = model.with_structured_output(Replan)

    result = replaner.invoke(messages)

    print(result, "replaner")

    if result.status == "done":
        return {"steps": [], "response": result.final_response}
    else:
        return {"steps": result.new_plan}


# langgraph图表创建
from langgraph.graph import StateGraph, START, END

workflow = StateGraph(AgentState)

workflow.add_node("planner", plan_node)

workflow.add_node("executor", execute_node)

workflow.add_node("replaner", replan_node)

workflow.add_edge(START, "planner")

workflow.add_edge("planner", "executor")

workflow.add_edge("executor", "replaner")


# 判断是否执行步骤还是结束
def should_end(state: AgentState):
    if "response" in state and state["response"]:
        return END
    else:
        return "executor"


workflow.add_conditional_edges("replaner", should_end)

app = workflow.compile()

In [ ]:
stream = app.stream(
    {"user_query": "帮我分析一下 ./data.csv 中的数据，并给出一个详细的数据报告"},
    stream_mode=["messages", "updates"],
)

for event in stream:
    print(event)